# Table of Contents
 <p><div class="lev1 toc-item"><a href="#User-story" data-toc-modified-id="User-story-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>User story</a></div><div class="lev1 toc-item"><a href="#Dataset-creation:" data-toc-modified-id="Dataset-creation:-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Dataset creation:</a></div><div class="lev2 toc-item"><a href="#Example-dataset-we-are-working-with:" data-toc-modified-id="Example-dataset-we-are-working-with:-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Example dataset we are working with:</a></div><div class="lev1 toc-item"><a href="#Queries-availables-through-the-query-api" data-toc-modified-id="Queries-availables-through-the-query-api-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Queries availables through the query api</a></div><div class="lev2 toc-item"><a href="#selected-point-table-view-of-all-variables-columns-" data-toc-modified-id="selected-point-table-view-of-all-variables-columns--31"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Selected point table view of all variables (columns)</a></div><div class="lev2 toc-item"><a href="#selected-point-table-view-of-selected-columns-filtering-by-date" data-toc-modified-id="selected-point-table-view-of-selected-columns-filtering-by-date-32"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Selected point table view of selected columns filtering by date</a></div><div class="lev2 toc-item"><a href="#selected-point-min-and-max-year-available" data-toc-modified-id="selected-point-min-and-max-year-available-33"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Selected point min and max year available</a></div><div class="lev2 toc-item"><a href="#selected-point-agg-statistics-for-selected-columns" data-toc-modified-id="selected-point-agg-statistics-for-selected-columns-34"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Selected point agg statistics for selected columns</a></div><div class="lev2 toc-item"><a href="#histograms" data-toc-modified-id="histograms-35"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Histograms</a></div>

In [5]:
import requests
import json
import pandas
from pprint import pprint
from IPython.display import display, Markdown, HTML

In [58]:
prod_env= 'https://api.resourcewatch.org'
staging_env='https://staging-api.globalforestwatch.org'
api_post= staging_env + '/v1/{endpoint}'
api_baseurl= staging_env + '/v1/{endpoint}/{dataset_id}'

def query(sql, dataset_id, geostore):
    display(Markdown(">#### Sql applied:  "))
    display(Markdown(">>```sql {0}  ```".format(sql)))
    payload = {"sql": sql,
              "geostore": geostore,
              "geoagg": "max"}
    r = requests.get(api_baseurl.format(endpoint='query',dataset_id=dataset_id), params=payload)
    if r.status_code != 200:
        return print(r.json())
    else:
        return pd.read_json(json.dumps(r.json()['data']), orient='records')

# User story

Dataset creation
* User should be able to create nexgddp datasets - as many as needed. (different time aggregation level, scenarios and models)

Widget editor:  
* User should be able to select a point in space (latitude + longitude). In the future we will allow to select a bounding box (in which case user needs to select the type of aggregation)
* User can filter the range of years from the available years. Available years will be implemented in the /fields response
* User can select type of chart (line or bar) and display it. Years should be ordered asc by default
* User has the Category, Value and Filters option (no order, no limit). 
* User can save the widget and embed it elsewhere

****Once Bounding Box selection is implemented:*** User can select different aggregation functions over the original data: avg, min, max, std, plus st_histogram and temporal series.


# Dataset creation:

```json
{
    "connectorType":"rest",
    "provider":"nexgddp",
    "tableName": "historical/ACCESS1_0"
    "application":[
     "rw"
    ],
    "name":"Nexgddp"
    ...
}
```
Dataset aggregated by year. The table name will be the `<scenario/model>` all in lowercase and `_` istead of `-`.  
F.E: `historical/access1_0`

List of scenarios
* historical
* rcp45
* rcp85

List of models available
* ACCESS1-0 
* CSIRO-MK3-6-0 
* MIROC-ESM
* BCC-CSM1-1 
* GFDL-CM3 
* MIROC-ESM-CHEM
* BNU-ESM 
* GFDL-ESM2G 
* MIROC5
* CanESM2 
* GFDL-ESM2M 
* MPI-ESM-LR
* CCSM4 
* INMCM4 
* MPI-ESM-MR
* CESM1-BGC 
* IPSL-CM5A-LR 
* MRI-CGCM3
* CNRM-CM5 
* IPSL-CM5A-MR 
* NorESM1-M


## Example dataset we are working with:

In [24]:
dataset_id='nexgddp-historical-ACCESS1_0'
geojson={"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"Point","coordinates":[-3.6749267578125,40.43858586704331]}}]}

r = requests.get(api_baseurl.format(endpoint='dataset',dataset_id=dataset_id))
pprint(r.json())    

{'data': {'attributes': {'application': ['prep'],
                         'attributesPath': None,
                         'blockchain': {},
                         'clonedHost': {},
                         'connectorType': 'rest',
                         'connectorUrl': '',
                         'dataPath': None,
                         'env': 'production',
                         'errorMessage': None,
                         'geoInfo': False,
                         'layerRelevantProps': [],
                         'legend': {'country': [], 'date': [], 'region': []},
                         'name': 'nexgddp-historical-ACCESS1_0',
                         'overwrite': False,
                         'provider': 'nexgddp',
                         'published': True,
                         'slug': 'nexgddp-historical-ACCESS1_0',
                         'status': 'saved',
                         'subtitle': None,
                         'tableName': 'historical/ACCESS1_

In [47]:
payload = {"geojson":geojson}
r = requests.post(api_post.format(endpoint='geostore'),json=payload)
geostore=r.json()['data']['id']

display(Markdown("""> #### geojson to display:  

> ```json 
{0} 
```""".format(geojson)))
display(Markdown("> ### Geostore creation:  "))
pprint(r.json())

> #### geojson to display:  

> ```json 
{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {}, 'geometry': {'type': 'Point', 'coordinates': [-3.6749267578125, 40.43858586704331]}}]} 
```

> ### Geostore creation:  

{'data': {'attributes': {'areaHa': 0,
                         'bbox': [-3.6749267578125,
                                  40.43858586704331,
                                  -3.6749267578125,
                                  40.43858586704331],
                         'geojson': {'crs': {},
                                     'features': [{'geometry': {'coordinates': [-3.6749267578125,
                                                                                40.43858586704331],
                                                                'type': 'Point'},
                                                   'type': 'Feature'}],
                                     'type': 'FeatureCollection'},
                         'hash': 'e669023a10c35364e0b1780c181bd06b',
                         'lock': False,
                         'provider': {}},
          'id': 'e669023a10c35364e0b1780c181bd06b',
          'type': 'geoStore'}}


In [34]:
r = requests.get(api_baseurl.format(endpoint='fields',dataset_id=dataset_id))
display(Markdown("> ### Dataset fields metadata: "))
pprint(r.json())

> ### Dataset fields metadata: 

{'fields': {'cdd': {'type': 'Quantity', 'uom': '10^0'},
            'cwd': {'type': 'Quantity', 'uom': '10^0'},
            'frost_days': {'type': 'Quantity', 'uom': '10^0'},
            'icing_days': {'type': 'Quantity', 'uom': '10^0'},
            'pr1p': {'type': 'Quantity', 'uom': '10^0'},
            'pr2in': {'type': 'Quantity', 'uom': '10^0'},
            'pr3in': {'type': 'Quantity', 'uom': '10^0'},
            'pr4in': {'type': 'Quantity', 'uom': '10^0'},
            'pr99p': {'type': 'Quantity', 'uom': '10^0'},
            'prmaxday': {'type': 'Quantity', 'uom': '10^0'},
            'tmax100F': {'type': 'Quantity', 'uom': '10^0'},
            'tmax1p': {'type': 'Quantity', 'uom': '10^0'},
            'tmax5day': {'type': 'Quantity', 'uom': '10^0'},
            'tmax90F': {'type': 'Quantity', 'uom': '10^0'},
            'tmax95F': {'type': 'Quantity', 'uom': '10^0'},
            'tmax99p': {'type': 'Quantity', 'uom': '10^0'},
            'tmin1p': {'type': 'Quantity', 'uom': '

In [33]:
r = requests.get(api_baseurl.format(endpoint='geostore',dataset_id=geostore))
display(Markdown("> ### Geostore"))
pprint(r.json())

> ### Geostore

{'data': {'attributes': {'areaHa': 0,
                         'bbox': [-3.6749267578125,
                                  40.43858586704331,
                                  -3.6749267578125,
                                  40.43858586704331],
                         'geojson': {'crs': {},
                                     'features': [{'geometry': {'coordinates': [-3.6749267578125,
                                                                                40.43858586704331],
                                                                'type': 'Point'},
                                                   'type': 'Feature'}],
                                     'type': 'FeatureCollection'},
                         'hash': 'e669023a10c35364e0b1780c181bd06b',
                         'lock': False,
                         'provider': {}},
          'id': 'e669023a10c35364e0b1780c181bd06b',
          'type': 'geoStore'}}


# Queries availables through the query api

It is mandatory to send a geostore representing a point (in the future also bbox will be allowed to evaluate a region of interest).  
In order to test it we are going to use the next geostore code: `e669023a10c35364e0b1780c181bd06b`

In [48]:
display(Markdown("> ## Selected point table view of all variables (columns)"))
sql = '''
select * from "historical/ACCESS1_0"
'''
dataset = query(sql, dataset_id, geostore)
display(dataset.head(6))
#display(Markdown("> ### Visual"))
#display(dataset.plot.bar(x='x',y='y',figsize=(10, 10)));

> ## Selected point table view of all variables (columns)

>#### Sql applied:  

>>```sql 
select * from "historical/ACCESS1_0"
  ```

{'errors': [{'detail': 'Invalid Select', 'status': 400}]}


AttributeError: 'NoneType' object has no attribute 'head'

In [57]:
display(Markdown("> ## Selected point table view of selected columns filtering by date"))
sql = '''
select tmax1p, year from nexgddp-historical-ACCESS1_0 where year between 1950 and 1955
'''
dataset = query(sql, dataset_id, geostore)
display(dataset.head(6))
#display(Markdown("> ### Visual"))
#display(dataset.plot.bar(x='x',y='y',figsize=(10, 10)));

> ## Selected point table view of selected columns filtering by date

>#### Sql applied:  

>>```sql 
select tmax1p, year from nexgddp-historical-ACCESS1_0 where year between 1950 and 1955
  ```

{'errors': [{'detail': 'Invalid Select', 'status': 400}]}


AttributeError: 'NoneType' object has no attribute 'head'

In [59]:
display(Markdown("> ## Selected point min and max year available"))
sql = '''
Select max(year), min(year) from from nexgddp-historical-ACCESS1_0
'''
dataset = query(sql, dataset_id, geostore)
display(dataset.head(6))
#display(Markdown("> ### Visual"))
#display(dataset.plot.bar(x='x',y='y',figsize=(10, 10)));

> ## Selected point min and max year available

>#### Sql applied:  

>>```sql 
Select max(year), min(year) from from nexgddp-historical-ACCESS1_0
  ```

{'errors': [{'detail': 'Invalid Select', 'status': 400}]}


AttributeError: 'NoneType' object has no attribute 'head'

In [69]:
display(Markdown("> ## Selected point agg statistics for selected columns"))
display(Markdown(""">  Available aggregation functions:
* avg(<column>)
* min(<column>)
* max(<column>)
* std(<column>) 

> ***`Group by <column>`  operation is not currently available*
"""))
sql = '''
Select avg(prmaxday) from nexgddp-historical-ACCESS1_0 where year between 1950 and 1955
'''
dataset = query(sql, dataset_id, geostore)
display(dataset.head(6))
#display(Markdown("> ### Visual"))
#display(dataset.plot.bar(x='x',y='y',figsize=(10, 10)));

> ## Selected point agg statistics for selected columns

>  Available aggregation functions:
* avg(<column>)
* min(<column>)
* max(<column>)
* std(<column>) 

> ***`Group by <column>`  operation is not currently available*


>#### Sql applied:  

>>```sql 
Select avg(prmaxday) from nexgddp-historical-ACCESS1_0 where year between 1950 and 1955
  ```

{'errors': [{'detail': 'Internal Server Error', 'status': 500}]}


AttributeError: 'NoneType' object has no attribute 'head'

In [71]:
display(Markdown("> ## Histograms"))
sql = '''
Select st_histogram(prmaxday) from nexgddp-historical-ACCESS1_0
'''
dataset = query(sql, dataset_id, geostore)
display(dataset.head(6))
#display(Markdown("> ### Visual"))
#display(dataset.plot.bar(x='x',y='y',figsize=(10, 10)));

> ## Histograms

>#### Sql applied:  

>>```sql 
Select st_histogram(prmaxday) from nexgddp-historical-ACCESS1_0
  ```

{'errors': [{'detail': 'Period of time must be set', 'status': 400}]}


AttributeError: 'NoneType' object has no attribute 'head'